In [1]:
import sys
!{sys.executable} -m pip install -U numpy matplotlib scapy[basic] tqdm

Requirement already up-to-date: numpy in c:\users\felix\appdata\local\programs\python\python38\lib\site-packages (1.18.1)
Requirement already up-to-date: matplotlib in c:\users\felix\appdata\local\programs\python\python38\lib\site-packages (3.1.3)
Requirement already up-to-date: scapy[basic] in c:\users\felix\appdata\local\programs\python\python38\lib\site-packages (2.4.3)
Requirement already up-to-date: tqdm in c:\users\felix\appdata\local\programs\python\python38\lib\site-packages (4.42.1)


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
import sys
import threading
from scapy.all import rdpcap
from tqdm.notebook import tqdm

sys.setrecursionlimit(100000)
threading.stack_size(200000000)

def load_captures(files):
    result = []
    
    for file in tqdm(files):
        # use threading to increase stack size
        thread = threading.Thread(target=lambda: result.append(rdpcap(file)))
        thread.start()
        thread.join()
        
    return result

In [3]:
files = ('capture-0-a4.pcap', 
         'capture-1-a4.pcap', 
         'capture-2-a4.pcap')

(commute_16122019, 
 commute_17122019,
 commute_18122019) = load_captures(files)

In [4]:
from scapy.all import Dot11

timestamps = []

for pkt in commute_16122019:
    if pkt.haslayer(Dot11) and pkt.type == 0 and pkt.subtype == 4:
        timestamps.append(pkt.time)